# Test NSEG dataset

In [1]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("./dataset_eeg_des.csv")
df.drop(['Unnamed: 0'],axis=1, inplace=True)
df = df.set_index(['id_user','session','video'])

In [3]:
df

0         1         2         3         4  \
id_user session video                                                     
10      1       1      2.135436  2.887686  3.465504  3.810436  3.955764   
                2      2.549241  2.254038  3.489210  3.876865  3.767604   
                3      2.436902  3.456774  4.163457  4.401829  4.272075   
                4      1.841611  2.513253  3.124816  3.421180  3.492676   
                5      2.320207  2.961560  3.857012  4.111162  3.868319   
...                         ...       ...       ...       ...       ...   
9       3       20     1.398441  2.604782  3.687751  3.648057  3.009953   
                21     1.571485  1.894192  2.786308  3.305627  3.571761   
                22     1.270886  2.200624  2.915375  3.471395  3.427353   
                23     2.621798  3.373337  3.905828  3.839850  3.868095   
                24     1.494128  3.076038  2.783606  3.572978  3.547337   

                              5         6         7         8         9  ...  \
id_user session video                                                    ...   
10      1       1      4.053275  3.646478  3.300221  3.606062  3.223825  ...   
                2      3.407996  3.706311  3.790671  3.835916  3.726078  ...   
                3      3.567253  3.214838  3.632954  3.966298  3.774067  ...   
                4      3.306529  3.099885  3.100102  3.221046  3.320871  ...   
                5      3.717754  3.698447  3.367067  3.021535  3.351307  ...   
...                         ...       ...       ...       ...       ...  ...   
9       3       20     3.942856  3.935395  2.950334  3.604765  3.776026  ...   
                21     3.696536  3.393062  2.770488  2.746998  2.790538  ...   
                22     2.811443  3.023582  2.642579  3.219103  3.161054  ...   
                23     3.649109  3.723545  3.259310  3.605300  3.623675  ...   
                24     2.896757  3.371674  3.948618  4.159415  4.232633  ...   

                            492       493       494       495       496  \
id_user session video                                                     
10      1       1     -1.011859 -1.020659 -0.602725 -0.584323 -0.602491   
                2     -0.654046 -0.287855 -0.064083 -0.382718 -0.854156   
                3     -0.102593 -0.080281 -0.138853  0.001005 -0.083322   
                4     -0.990079 -0.927056 -0.745584 -0.978420 -1.031638   
                5     -0.247413 -0.758625 -1.643860 -1.252678 -0.605932   
...                         ...       ...       ...       ...       ...   
9       3       20    -0.389243 -0.347923 -0.460666 -0.483391 -0.797307   
                21    -0.804578 -1.045797 -1.104529 -0.949304 -0.719607   
                22    -0.531922 -0.662631 -0.867511 -1.125926 -0.903615   
                23    -0.544234 -0.343033 -0.551949 -0.976702 -1.112032   
                24    -0.602491  0.003543  0.088560 -0.193022 -0.742112   

                            497       498       499       500  emotion  
id_user session video                                                   
10      1       1     -0.579400 -1.200410 -0.910379 -1.369580        1  
                2     -0.538963 -0.978321 -0.520367 -1.108300        2  
                3     -0.067288 -0.063967  0.042125 -0.123110        3  
                4     -0.682465 -0.665910 -1.033460 -1.373817        0  
                5     -0.750933 -1.202377 -0.686867 -1.160245        2  
...                         ...       ...       ...       ...      ...  
9       3       20    -0.593601 -0.401147 -0.498099 -0.751132        0  
                21    -0.668664 -0.650504 -0.838495 -1.361561        2  
                22    -1.054831 -0.916811 -1.142544 -1.251611        0  
                23    -0.834240 -0.739685 -0.623760 -1.128035        1  
                24    -1.026718 -0.977749 -0.960097 -1.363363        0  

[1080 rows x 502 columns]

# Subject dependent

In [4]:
df_sd = df.reset_index()

In [5]:
df_sd1 = df_sd[df_sd.session == 1]

In [6]:
np.random.seed(75)
test_videos = np.random.choice(np.arange(1, 25), replace=False, size=(8))
print(test_videos)

[ 2 22 12 14  6 11 24 21]


In [7]:
X_test_SD1 = df_sd1[df_sd1['video'].isin(test_videos)].set_index(['id_user','session','video']).drop('emotion',axis=1)

In [8]:
y_test_SD1 = df_sd1[df_sd1['video'].isin(test_videos)].set_index(['id_user','session','video']).emotion

In [9]:
X_train_SD1 = df_sd1[~df_sd1['video'].isin(test_videos)].set_index(['id_user','session','video']).drop('emotion',axis=1)

In [10]:
y_train_SD1 = df_sd1[~df_sd1['video'].isin(test_videos)].set_index(['id_user','session','video']).emotion

In [11]:
X_train_SD1 = np.array(X_train_SD1).reshape((X_train_SD1.shape[0], X_train_SD1.shape[1], 1))

In [12]:
y_train_SD1 = to_categorical(y_train_SD1)

In [13]:
X_test_SD1 = np.array(X_test_SD1).reshape((X_test_SD1.shape[0], X_test_SD1.shape[1], 1))

In [14]:
y_test_SD1 = to_categorical(y_test_SD1)

In [15]:
modelSD = Sequential()

In [16]:
modelSD.add(LSTM(units=64, input_shape=(X_train_SD1.shape[1], 1)))

In [17]:
modelSD.add(Dense(units=4, activation='relu'))

In [18]:
modelSD.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [19]:
modelSD.fit(X_train_SD1, y_train_SD1, epochs=50, batch_size=64)

Epoch 1/50
4/4 [==============================] - 8s 650ms/step - loss: 7.8796 - accuracy: 0.2417
Epoch 2/50
4/4 [==============================] - 3s 616ms/step - loss: 8.1840 - accuracy: 0.2458
Epoch 3/50
4/4 [==============================] - 3s 652ms/step - loss: 7.9649 - accuracy: 0.2542
Epoch 4/50
4/4 [==============================] - 3s 656ms/step - loss: 7.9455 - accuracy: 0.2583
Epoch 5/50
4/4 [==============================] - 2s 631ms/step - loss: 7.8393 - accuracy: 0.2583
Epoch 6/50
4/4 [==============================] - 3s 663ms/step - loss: 7.8295 - accuracy: 0.2708
Epoch 7/50
4/4 [==============================] - 3s 752ms/step - loss: 7.5721 - accuracy: 0.2708
Epoch 8/50
4/4 [==============================] - 3s 683ms/step - loss: 6.2209 - accuracy: 0.2917
Epoch 9/50
4/4 [==============================] - 3s 657ms/step - loss: 4.5216 - accuracy: 0.3083
Epoch 10/50
4/4 [==============================] - 3s 665ms/step - loss: 4.5368 - accuracy: 0.2458
Epoch 11/50
4/4 [==

In [20]:
loss, accuracy = modelSD.evaluate(X_test_SD1, y_test_SD1)

4/4 [==============================] - 2s 133ms/step - loss: 1.3592 - accuracy: 0.3750


# Subject-Biased Experiment


In [21]:
X_trainSB, X_testSB, y_trainSB, y_testSB = train_test_split(df.drop('emotion',axis=1), df.emotion, test_size=0.20, random_state=42)

In [22]:
X_trainSB = np.array(X_trainSB).reshape((X_trainSB.shape[0], X_trainSB.shape[1], 1))

In [23]:
y_trainSB = to_categorical(y_trainSB)

In [24]:
X_testSB = np.array(X_trainSB).reshape((X_trainSB.shape[0], X_trainSB.shape[1], 1))

In [25]:
y_testSB = to_categorical(y_testSB)

In [26]:
modelSB = Sequential()

In [27]:
modelSB.add(LSTM(units=64, input_shape=(X_trainSB.shape[1], 1)))

In [28]:
modelSB.add(Dense(units=4, activation='tanh'))

In [29]:
modelSB.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [30]:
modelSB.fit(X_trainSB, y_trainSB, epochs=10, batch_size=32)

Epoch 1/10
27/27 [==============================] - 12s 316ms/step - loss: 7.7324 - accuracy: 0.2442
Epoch 2/10
27/27 [==============================] - 9s 326ms/step - loss: 6.9752 - accuracy: 0.2512
Epoch 3/10
27/27 [==============================] - 9s 325ms/step - loss: 7.2320 - accuracy: 0.2523
Epoch 4/10
27/27 [==============================] - 9s 325ms/step - loss: 8.5073 - accuracy: 0.2523
Epoch 5/10
27/27 [==============================] - 8s 311ms/step - loss: 7.5768 - accuracy: 0.2558
Epoch 6/10
27/27 [==============================] - 8s 306ms/step - loss: 7.1311 - accuracy: 0.2512
Epoch 7/10
27/27 [==============================] - 9s 324ms/step - loss: 4.5627 - accuracy: 0.2558
Epoch 8/10
27/27 [==============================] - 9s 328ms/step - loss: 9.2375 - accuracy: 0.2546
Epoch 9/10
27/27 [==============================] - 9s 317ms/step - loss: 7.7607 - accuracy: 0.2546
Epoch 10/10
27/27 [==============================] - 8s 305ms/step - loss: 7.7262 - accuracy: 0.254

In [31]:
loss, accuracy = modelSB.evaluate(X_testSB, y_testSB)

ValueError: Data cardinality is ambiguous:
  x sizes: 864
  y sizes: 216
Make sure all arrays contain the same number of samples.

# Subject Indipendent

In [ ]:
def subject_indipendent_split():
    df_SI = df.reset_index()
    groups = df_SI['id_user']
    X_SI = df_SI.drop(['emotion','id_user','session','video'],axis=1)
    y_SI = df_SI.emotion
    return X_SI,y_SI,groups